# Installs e imports

In [17]:
!pip install groq

In [18]:
import os
from groq import Groq
from google.colab import userdata
import re
import requests

# Teste Inicial

Uso de um completion básico para testar a funcionalidade da API do Groq e a key que será utilizada

In [19]:
client = Groq(api_key=userdata.get('GROQ_API_KEY'))

chat_completion = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Explain the importance of fast language models"}
    ],
    model="llama3-70b-8192",
    temperature=0
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's natural language processing (NLP) landscape, and their importance can be seen in several aspects:

1. **Real-time Applications**: Fast language models enable real-time applications such as chatbots, virtual assistants, and language translation systems to respond quickly and efficiently. This is particularly important in customer-facing applications where delayed responses can lead to frustration and a negative user experience.
2. **Low-Latency Requirements**: Many applications, such as voice assistants, require language models to process and respond to user input within a few hundred milliseconds. Fast language models can meet these low-latency requirements, ensuring a seamless user experience.
3. **Scalability**: Fast language models can handle large volumes of data and user requests, making them essential for large-scale NLP applications such as language translation, sentiment analysis, and text summarization.
4. **Energy Efficiency**: Fast

# Setup da Classe do Agente

* Inicialização (__init__): Configura o agente com um cliente (para acessar a
API) e opcionalmente uma mensagem de sistema, que pode ser utilizada para fornecer contexto ou instruções iniciais ao modelo.  

* Chamada do Agente (__call__): Permite usar a instância do agente como se fosse uma função. Quando chamada, adiciona uma nova mensagem do usuário ao histórico, solicita a resposta do modelo via o método execute, registra essa resposta no histórico como uma mensagem do assistente e, por fim, retorna a resposta.  

* Execução da Chamada à API (execute): Envia o histórico de mensagens para o modelo de linguagem fazer um completion.

In [20]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = self.client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content

# Prompt

Define o comportamento do agente, orientando-o a seguir um ciclo ReAct (Thought → Action → PAUSE → Observation) até chegar à resposta final.  

**Estrutura ReAct:**
* Thought: O agente descreve seu raciocínio.
* Action: O agente executa uma ação (por exemplo, realizar um cálculo ou buscar a massa de um planeta).
* PAUSE: Uma indicação para esperar o resultado da ação executada.
* Observation: O agente recebe o resultado da ação e utiliza essa informação para continuar o raciocínio.  

**Ações/Tools Disponíveis:**  
* calculate: para operações matemáticas.  
* get_planet_mass: para recuperar a massa de um planeta.

## Ajuste no Prompt

Foram adicionadas duas novas tools que ampliam a capacidade do agente, permitindo que ele forneça respostas mais ricas e informativas. Assim, além de realizar cálculos, o agente pode acessar dados detalhados (por exemplo, características físicas e históricas dos planetas), tornando a resposta final mais completa e contextualizada.  

Além disso, após testes práticos, notei que o agente estava respondendo apenas a parte dos cálculos, ignorando a solicitação de apresentar um fato interessante. Logo, criei uma nova instrução, em que o agente é explicitamente orientado a integrar todas as informações solicitadas, garantindo que a resposta final aborde tanto as curiosidades quanto os resultados dos cálculos.

In [21]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

If your question includes multiple aspects (for example, asking for interesting
facts and performing calculations), your final Answer must incorporate all relevant information in a single, cohesive response.
Specifically, include both the interesting fact and the result of the calculation.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary.

get_planet_mass:
e.g. get_planet_mass: Earth
Returns the mass of the planet in kg.

get_planet_radius:
e.g. get_planet_radius: Earth
Returns the radius of the planet in kilometers.

get_planet_info:
e.g. get_planet_info: Earth
Dynamically fetches interesting historical facts or curiosities about the planet from Wikipedia.

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Another example:

Question: Tell me something interesting about Mars.
Thought: I need to fetch historical facts about Mars.
Action: get_planet_info: Mars
PAUSE

You will be called again with this:

Observation: [Wikipedia summary about Mars]

If you have the answer, output it as the Answer.

Answer: Mars is fascinating because [summary].

Now it's your turn:
""".strip()

# Funções/Tools do Agente

**calculate:**  
* Avalia uma string que contém uma operação matemática (por exemplo, "4 * 7 / 3") usando a função eval e retorna o resultado numérico.

**get_planet_mass:**  
* Recebe o nome de um planeta, padroniza o texto para minúsculas e, usando um match-case, retorna a massa do planeta correspondente.  
* Se o planeta não estiver listado, retorna 0.0, indicando que o planeta não foi reconhecido.

In [22]:
def calculate(operation: str) -> float:
    return eval(operation)


def get_planet_mass(planet) -> float:
    match planet.lower():
        case "earth":
            return 5.972e24
        case "jupiter":
            return 1.898e27
        case "mars":
            return 6.39e23
        case "mercury":
            return 3.285e23
        case "neptune":
            return 1.024e26
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "venus":
            return 4.867e24
        case _:
            return 0.0

## Novas Tools

Visando aumentar as capacidades do agente, forneci duas novas tools que permitem buscar mais informações (vindas da Wikipedia) sobre os planetas e também fornecer o raio em kms de alguns planetas.

**get_planet_info:**

* Recebe o nome de um planeta, formata-o para corresponder ao padrão da Wikipedia e realiza uma requisição HTTP à API REST da Wikipedia para buscar um resumo da página correspondente ao planeta. Se a requisição for bem-sucedida, retorna o resumo com informações históricas e curiosidades, caso contrário, retorna uma mensagem de erro ou uma indicação de que a informação não foi encontrada.

**get_planet_radius:**

* Recebe o nome de um planeta, converte-o para letras minúsculas e utiliza uma estrutura match-case para retornar o raio do planeta em quilômetros. Se o planeta não estiver listado, retorna 0.0, indicando que o planeta não foi reconhecido.

In [23]:
def get_planet_info(planet: str) -> str:
    """
    Busca informações dinâmicas sobre o planeta na Wikipedia.
    Utiliza a API REST para obter um resumo da página correspondente ao planeta.

    Parâmetros:
      planet (str): Nome do planeta a ser consultado.

    Retorna:
      str: Resumo extraído da página da Wikipedia ou uma mensagem de erro.
    """
    try:
        # Formata o nome do planeta
        planet_title = planet.capitalize()
        # URL da API REST da Wikipedia para obter o resumo de uma página
        url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{planet_title}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            # Retorna o resumo extraído da resposta, ou uma mensagem padrão se não houver resumo.
            return data.get("extract", "No summary available.")
        else:
            return f"Error: Received status code {response.status_code} from Wikipedia."
    except Exception as e:
        return f"Error: {e}"

In [24]:
def get_planet_radius(planet) -> float:
    """
    Retorna o raio do planeta em quilômetros.
    """
    match planet.lower():
        case "earth":
            return 6371
        case "jupiter":
            return 69911
        case "mars":
            return 3389.5
        case "mercury":
            return 2439.7
        case "neptune":
            return 24622
        case "saturn":
            return 58232
        case "uranus":
            return 25362
        case "venus":
            return 6051.8
        case _:
            return 0.0

# Agente com loop ("automático")

**Instanciação do Agente:**  
A função cria uma instância de Agent passando o client e o system_prompt previamente definidos.

**Definição das Ferramentas:**  
Uma lista de ferramentas disponíveis (calculate e get_planet_mass) é utilizada para determinar quais ações o agente pode pedir.

**Loop:**  
Envia um prompt para o agente e printa sua resposta. Se a resposta incluir uma ação (indicada por "Action" e "PAUSE"), o código extrai a ação e seu argumento utilizando expressões regulares.

Se a ação for válida (estiver na lista de ferramentas), a função correspondente é executada (por exemplo, calcular uma expressão ou obter a massa de um planeta) e o resultado é enviado como observação ao agente.  
Se a resposta contiver "Answer", o loop é interrompido.

Ou seja, essa função automatiza o processo que foi feito de forma manual anteriormente, em que o agente "pedia" para usar uma ação, nós fazíamos a execução e depois apenas o resultado era passado de volta ao agente.

In [25]:
""" Aumento do número de interações máximas, visto que agora o agente é capaz de
responder diversas solicitações (ex: fatos sobre o planeta, raio e cálculo de massas)
em uma mesma query, o que demanda mais interações.
"""
def loop(max_iterations=20, query: str = ""):

    agent = Agent(client=client, system=system_prompt)

    tools = ["calculate", "get_planet_mass", "get_planet_radius", "get_planet_info"]

    next_prompt = query

    i = 0

    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break


loop(query="Tell me something interesting about Jupiter, then calculate its mass plus the mass of Saturn, and multiply all of that by 2.")

Thought: I need to fetch historical facts about Jupiter and then perform some calculations involving Jupiter and Saturn's masses.

Action: get_planet_info: Jupiter
PAUSE
Observation: Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass more than 2.5 times that of all the other planets in the Solar System combined and slightly less than one-thousandth the mass of the Sun. Its diameter is eleven times that of Earth, and a tenth that of the Sun. Jupiter orbits the Sun at a distance of 5.20 AU (778.5 Gm), with an orbital period of 11.86 years. It is the third-brightest natural object in the Earth's night sky, after the Moon and Venus, and has been observed since prehistoric times. Its name derives from that of Jupiter, the chief deity of ancient Roman religion.
Thought: That's an interesting fact about Jupiter. Now, I need to get the masses of Jupiter and Saturn to perform the calculations.

Action: get_planet_mass: Jupiter
PAUSE
Obser